In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from tqdm import tqdm


In [2]:
pd.set_option('display.max_row',15)
pd.set_option('display.max_column',None)

# Part1: loading data

## 1) IUCN: habitat breadth

In [3]:

pa_habitat = pd.read_csv('./IUCN_passeri/habitats.csv')
nonpa_habitat = pd.read_csv('./IUCN_no_passeri/habitats.csv')
habitat_df = pd.concat([
    pa_habitat,
    nonpa_habitat
], axis=0)
a = habitat_df[['scientificName','code']]
a['value'] = 1
habitat_matrix = a.pivot_table(index='scientificName', columns='code').fillna(0).droplevel(axis=1, level=0)


In [4]:
hb_df = []
for index,line in tqdm(habitat_matrix.iterrows(), total = len(habitat_matrix)):
    hat_types = list(line[line==1].index)
    res = []
    for hat_type in hat_types:
        co_sp = list(habitat_matrix.index[habitat_matrix[hat_type]==1])
        res.append(co_sp)
    total = set()
    for i in res:
        total.update(i)
    habitat_breadth = len(total) / np.mean([len(i) for i in res])
    hb_df.append({
        'sp':index,
        'habitat_breadth':habitat_breadth
    })
    
    

100%|██████████| 11188/11188 [00:06<00:00, 1690.03it/s]


In [33]:
habitat_df = pd.DataFrame(hb_df).rename(columns={'sp':'Species1_BirdLife'})
habitat_df

,Species1_BirdLife,habitat_breadth
0,Abeillia abeillei,1.432197
1,Abroscopus albogularis,1.618039
2,Abroscopus schisticeps,1.000000
3,Abroscopus superciliaris,1.473197
4,Aburria aburri,1.832535
...,...,...
11183,Zosterornis hypogrammicus,1.000000
11184,Zosterornis latistriatus,1.000000
11185,Zosterornis nigrorum,1.759202
11186,Zosterornis striatus,2.294362


## 2) IUCN: redlist categories & realm


In [39]:
nonpa_IUCN = pd.read_csv('./IUCN_no_passeri/assessments.csv')
pa_IUCN = pd.read_csv('./IUCN_passeri/assessments.csv')
IUCN_df = pd.concat([
    nonpa_IUCN,
    pa_IUCN
], axis=0)
IUCN_df = IUCN_df[['scientificName','redlistCategory','populationTrend','realm']].reset_index(drop=True)

In [40]:
redlistCategory_dict = {
    'Least Concern':1, 
    'Near Threatened':2,
    'Vulnerable':3,
    'Endangered':4, 
    'Critically Endangered':5,
    'Extinct in the Wild':6,
    'Extinct':7,
    'Data Deficient': np.nan
}

populationTrend_dict = {
    'Decreasing':1,
    'Stable':2,
    'Increasing':3,
    'Unknown':np.nan,
    np.nan:np.nan
}

from collections import Counter
Real_df = pd.DataFrame({k:Counter(['Realm_'+jj for jj in i.split('|')]) for k,i in zip(IUCN_df.scientificName,IUCN_df.realm)}).T.fillna(0).reset_index(drop=False)


In [41]:
IUCN_df = pd.concat([
    Real_df.rename(columns={'index':'Species1_BirdLife'}),
    IUCN_df['redlistCategory'].apply(lambda x:redlistCategory_dict[x]),
    IUCN_df['populationTrend'].apply(lambda x:populationTrend_dict[x]),
], axis=1)
IUCN_df

,Species1_BirdLife,Realm_Indomalayan,Realm_Palearctic,Realm_Nearctic,Realm_Afrotropical,Realm_Oceanian,Realm_Neotropical,Realm_Australasian,Realm_Antarctic,redlistCategory,populationTrend
0,Polyplectron schleiermacheri,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,1.0
1,Tetraogallus caspius,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
2,Alectoris barbara,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
3,Ammoperdix griseogularis,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0
4,Phasianus colchicus,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
11183,Rhipidura personata,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,1.0
11184,Calyptomena viridis,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0
11185,Iole finschii,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0
11186,Zimmerius chrysops,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,2.0


## 3) EltonTraits: Diet & Foraging

In [93]:
diet_forage_df = pd.read_csv('./EltonTraits/BirdFuncDat.txt',sep='\t',encoding='unicode_escape')

In [94]:
diet_forage_df = diet_forage_df[['English','Scientific',
                'Diet-Inv', 'Diet-Vend',
       'Diet-Vect', 'Diet-Vfish', 'Diet-Vunk', 'Diet-Scav', 'Diet-Fruit',
       'Diet-Nect', 'Diet-Seed', 'Diet-PlantO', 'Diet-5Cat',
       'ForStrat-watbelowsurf',
       'ForStrat-wataroundsurf', 'ForStrat-ground', 'ForStrat-understory',
       'ForStrat-midhigh', 'ForStrat-canopy', 'ForStrat-aerial',
                ]].rename(columns={'English':'Common_name','Scientific':'Species1_BirdLife'})
diet_forage_df = diet_forage_df.dropna(axis=0,how='all')
diet_forage_df.columns = [i.replace('-','_') for i in diet_forage_df.columns]
a = pd.get_dummies(diet_forage_df['Diet_5Cat'])
a.columns = ['Diet_5Cat'+i for i in a.columns]
diet_forage_df = pd.concat([
    diet_forage_df[['Species1_BirdLife']],
    diet_forage_df[[i for i in diet_forage_df.columns if 'Diet' in i]],
    a,
    diet_forage_df[[i for i in diet_forage_df.columns if 'ForStrat' in i]]
], axis=1).drop('Diet_5Cat', axis=1)
diet_forage_df['Diet_Breadth'] = diet_forage_df[['Diet_Inv', 'Diet_Vend',
       'Diet_Vect', 'Diet_Vfish', 'Diet_Vunk', 'Diet_Scav', 'Diet_Fruit',
       'Diet_Nect', 'Diet_Seed', 'Diet_PlantO']].applymap(lambda x:1 if x>0 else 0).sum(axis=1)
diet_forage_df['ForStrat_Breadth'] = diet_forage_df[['ForStrat_watbelowsurf',
       'ForStrat_wataroundsurf', 'ForStrat_ground', 'ForStrat_understory',
       'ForStrat_midhigh', 'ForStrat_canopy', 'ForStrat_aerial']].applymap(lambda x:1 if x>0 else 0).sum(axis=1)
diet_forage_df

,Species1_BirdLife,Diet_Inv,Diet_Vend,Diet_Vect,Diet_Vfish,Diet_Vunk,Diet_Scav,Diet_Fruit,Diet_Nect,Diet_Seed,Diet_PlantO,Diet_5CatFruiNect,Diet_5CatInvertebrate,Diet_5CatOmnivore,Diet_5CatPlantSeed,Diet_5CatVertFishScav,ForStrat_watbelowsurf,ForStrat_wataroundsurf,ForStrat_ground,ForStrat_understory,ForStrat_midhigh,ForStrat_canopy,ForStrat_aerial,Diet_Breadth,ForStrat_Breadth
0,Struthio camelus,10.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,30.0,50.0,0,0,0,1,0,0.0,0.0,100.0,0.0,0.0,0.0,0.0,4,1
1,Rhea americana,20.0,10.0,10.0,0.0,0.0,0.0,20.0,0.0,20.0,20.0,0,0,1,0,0,0.0,0.0,100.0,0.0,0.0,0.0,0.0,6,1
2,Rhea pennata,20.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,30.0,40.0,0,0,0,1,0,0.0,0.0,100.0,0.0,0.0,0.0,0.0,4,1
3,Casuarius casuarius,10.0,0.0,0.0,0.0,10.0,0.0,80.0,0.0,0.0,0.0,1,0,0,0,0,0.0,0.0,80.0,20.0,0.0,0.0,0.0,3,2
4,Casuarius bennetti,10.0,0.0,0.0,0.0,10.0,0.0,80.0,0.0,0.0,0.0,1,0,0,0,0,0.0,0.0,80.0,20.0,0.0,0.0,0.0,3,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9988,Zosterops somadikartai,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0,0,0,0.0,0.0,0.0,50.0,50.0,0.0,0.0,1,2
9989,Hirundo perdita,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0,0,0,0.0,0.0,0.0,50.0,50.0,0.0,0.0,1,2
9990,Cinclodes olrogi,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0,0,0,0.0,0.0,100.0,0.0,0.0,0.0,0.0,1,1
9991,Xiphorhynchus elegans,90.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0,0,0,0.0,0.0,0.0,30.0,60.0,10.0,0.0,2,3


## 4) AVONET - body functional traits


In [95]:
AVONET_raw = pd.read_csv('./AVONET/AVONET_Raw_Data.csv')
AVONET_birdlife = pd.read_csv('./AVONET/AVONET1_BirdLife.csv')

In [104]:
AVONET_df = AVONET_raw[['Species1_BirdLife','Species2_eBird','Species3_BirdTree']].drop_duplicates().merge(
    AVONET_birdlife[[
     'Species1',
     'Beak.Length_Culmen', 'Beak.Length_Nares', 'Beak.Width', 'Beak.Depth',
       'Tarsus.Length', 'Wing.Length', 'Kipps.Distance', 'Secondary1',
       'Hand-Wing.Index', 'Tail.Length', 'Mass',
       'Habitat', 'Habitat.Density', 'Migration', 'Trophic.Level',
       'Trophic.Niche', 'Primary.Lifestyle', 'Min.Latitude', 'Max.Latitude',
       'Centroid.Latitude', 'Centroid.Longitude', 'Range.Size'
]].rename(columns={'Species1':'Species1_BirdLife'}), 
    left_on='Species1_BirdLife', right_on='Species1_BirdLife', how='left'
)

a = pd.get_dummies(AVONET_df['Trophic.Level'])
a.columns = ['Trophic_Level_'+i for i in a.columns]
b = pd.get_dummies(AVONET_df['Trophic.Niche'])
b.columns = ['Trophic_Niche_'+i for i in b.columns]
c = pd.get_dummies(AVONET_df['Primary.Lifestyle'])
c.columns = ['Primary_Lifestyle_'+i for i in c.columns]

AVONET_df = pd.concat(
    [
        AVONET_df.drop(['Habitat','Habitat.Density','Trophic.Level','Trophic.Niche','Primary.Lifestyle'], axis=1),
        a,b,c
    ], axis=1
)
AVONET_df.columns = [i.replace('.','_') for i in AVONET_df.columns]
AVONET_df

,Species1_BirdLife,Species2_eBird,Species3_BirdTree,Beak_Length_Culmen,Beak_Length_Nares,Beak_Width,Beak_Depth,Tarsus_Length,Wing_Length,Kipps_Distance,Secondary1,Hand-Wing_Index,Tail_Length,Mass,Migration,Min_Latitude,Max_Latitude,Centroid_Latitude,Centroid_Longitude,Range_Size,Trophic_Level_Carnivore,Trophic_Level_Herbivore,Trophic_Level_Omnivore,Trophic_Level_Scavenger,Trophic_Niche_Aquatic predator,Trophic_Niche_Frugivore,Trophic_Niche_Granivore,Trophic_Niche_Herbivore aquatic,Trophic_Niche_Herbivore terrestrial,Trophic_Niche_Invertivore,Trophic_Niche_Nectarivore,Trophic_Niche_Omnivore,Trophic_Niche_Scavenger,Trophic_Niche_Vertivore,Primary_Lifestyle_Aerial,Primary_Lifestyle_Aquatic,Primary_Lifestyle_Generalist,Primary_Lifestyle_Insessorial,Primary_Lifestyle_Terrestrial
0,Abeillia abeillei,Abeillia abeillei,Abeillia abeillei,13.0,9.5,1.2,1.7,6.3,47.0,30.5,15.5,66.3,29.4,2.7,1.0,12.78,19.77,15.29,-89.79,144610.40,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0
1,Abroscopus albogularis,Abroscopus albogularis,Abroscopus albogularis,9.0,4.9,2.7,2.3,15.3,43.4,5.7,37.3,13.3,38.7,4.8,1.0,14.97,31.56,24.78,105.57,1704650.60,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0
2,Abroscopus schisticeps,Abroscopus schisticeps,Abroscopus schisticeps,9.0,4.7,2.6,2.3,16.5,47.1,7.6,39.5,16.1,47.5,4.7,1.0,20.19,31.12,26.70,95.58,909217.05,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0
3,Abroscopus superciliaris,Abroscopus superciliaris,Abroscopus superciliaris,12.3,7.4,3.1,2.8,16.6,51.2,6.5,44.5,12.7,42.4,6.5,1.0,-8.11,28.81,9.89,105.20,2493437.47,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0
4,Aburria aburri,Aburria aburri,Aburria aburri,37.1,16.6,9.3,10.2,65.8,337.2,52.5,283.1,15.7,290.4,1405.1,1.0,-13.43,11.15,-2.79,-75.23,139314.36,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11396,Zosterornis hypogrammicus,Zosterornis hypogrammicus,Stachyris hypogrammica,17.7,9.7,3.8,5.0,22.6,71.5,12.6,58.9,17.6,60.3,21.6,1.0,8.76,9.50,9.08,117.96,256.97,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0
11397,Zosterornis latistriatus,Zosterornis latistriatus,Stachyris latistriata,21.0,11.4,4.0,5.2,24.9,67.0,8.4,58.6,12.5,59.0,29.5,1.0,10.95,11.43,11.21,122.22,285.09,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0
11398,Zosterornis nigrorum,Zosterornis nigrorum,Stachyris nigrorum,15.2,8.8,3.4,3.8,22.3,68.2,10.9,57.3,16.1,49.6,21.6,1.0,9.09,10.51,9.52,123.12,468.62,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0
11399,Zosterornis striatus,Zosterornis striatus,Stachyris striata,14.6,9.0,3.2,3.8,17.8,62.1,9.5,52.6,15.2,51.2,13.9,1.0,13.96,18.58,16.15,121.76,20400.67,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0


## 5) Brain_size

In [120]:
Brain_size_df = pd.read_csv('./Brain_size/Brain_size.csv')
Brain_size_df = Brain_size_df[['Species','BrainSize','RelativeBrainSize']].rename(columns={'Species':'Species1_BirdLife'})
Brain_size_df['Species1_BirdLife'] = [i.replace('_',' ') for i in Brain_size_df['Species1_BirdLife']]

In [121]:
Brain_size_df

,Species1_BirdLife,BrainSize,RelativeBrainSize
0,Abroscopus albogularis,0.32,0.251
1,Acanthagenys rufogularis,1.16,0.273
2,Acanthiza chrysorrhoa,0.43,0.192
3,Acanthiza ewingii,0.63,1.053
4,Acanthiza inornata,0.44,0.272
...,...,...,...
1926,Zoothera heinei,1.80,0.475
1927,Zoothera lunulata,2.16,0.458
1928,Zoothera naevia,1.71,0.389
1929,Zosterops japonicus,0.54,0.375


## 6) Innovativeness

In [113]:
Inno_df = pd.read_csv('./Innovativeness/Innovativeness.csv')

In [114]:
Inno_df = Inno_df[['Species','GenerationLength','TotalInnovations',
         'TotalInnovations', 'TotalFoodTypeInnovations',
       'TotalTechnicalInnovations', 'InnovationYesOrNo',
       'FoodTypeInnovationYesOrNo', 'TechnicalInnovationYesOrNo']].rename(columns={'Species':'Species1_BirdLife'})
Inno_df['Species1_BirdLife'] = [i.replace('_',' ') for i in Inno_df['Species1_BirdLife']]


In [115]:
Inno_df

,Species1_BirdLife,GenerationLength,TotalInnovations,TotalInnovations,TotalFoodTypeInnovations,TotalTechnicalInnovations,InnovationYesOrNo,FoodTypeInnovationYesOrNo,TechnicalInnovationYesOrNo
0,Abeillia abeillei,4.2,0,0,0,0,0,0,0
1,Aburria aburri,5.7,0,0,0,0,0,0,0
2,Acanthagenys rufogularis,6.8,1,1,1,0,1,1,0
3,Acanthisitta chloris,3.5,1,1,1,0,1,1,0
4,Acanthiza apicalis,5.7,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
8640,Vanellus lugubris,8.9,0,0,0,0,0,0,0
8641,Vanellus superciliosus,8.9,0,0,0,0,0,0,0
8642,Vermivora bachmanii,3.8,0,0,0,0,0,0,0
8643,Vireo altiloquus,4.2,0,0,0,0,0,0,0


## 7) eBird Taxonomy

In [116]:
ebird_taxomony_df = pd.read_csv('./ebird_taxonomy/eBird_Taxonomy_v2021.csv')
ebird_taxomony_df = ebird_taxomony_df[ebird_taxomony_df.CATEGORY=='species']
ebird_taxomony_df = ebird_taxomony_df[[
    'SCI_NAME','PRIMARY_COM_NAME','ORDER1','FAMILY',
]].rename(columns={
    'PRIMARY_COM_NAME':'Common_Name1_eBird',
    'SCI_NAME':'Species2_eBird',
    'ORDER1':'ORDER1_eBird',
    'FAMILY':'FAMILY1_eBird'
})
ebird_taxomony_df

,Species2_eBird,Common_Name1_eBird,ORDER1_eBird,FAMILY1_eBird
0,Struthio camelus,Common Ostrich,Struthioniformes,Struthionidae (Ostriches)
1,Struthio molybdophanes,Somali Ostrich,Struthioniformes,Struthionidae (Ostriches)
3,Rhea americana,Greater Rhea,Rheiformes,Rheidae (Rheas)
4,Rhea pennata,Lesser Rhea,Rheiformes,Rheidae (Rheas)
7,Nothocercus julius,Tawny-breasted Tinamou,Tinamiformes,Tinamidae (Tinamous)
...,...,...,...,...
16745,Saltator maxillosus,Thick-billed Saltator,Passeriformes,Thraupidae (Tanagers and Allies)
16746,Saltator aurantiirostris,Golden-billed Saltator,Passeriformes,Thraupidae (Tanagers and Allies)
16747,Saltator cinctus,Masked Saltator,Passeriformes,Thraupidae (Tanagers and Allies)
16748,Saltator grossus,Slate-colored Grosbeak,Passeriformes,Thraupidae (Tanagers and Allies)


# Part2: Merging

In [124]:
all_combined_traits_df = ebird_taxomony_df.merge(
    AVONET_df, left_on='Species2_eBird', right_on='Species2_eBird', how='left'
).merge(
    IUCN_df, left_on='Species1_BirdLife', right_on='Species1_BirdLife', how='left'
).merge(
    habitat_df, left_on='Species1_BirdLife', right_on='Species1_BirdLife', how='left'
).merge(
    diet_forage_df, left_on='Species1_BirdLife', right_on='Species1_BirdLife', how='left'
).merge(
    Inno_df, left_on='Species1_BirdLife', right_on='Species1_BirdLife', how='left'
).merge(
    Brain_size_df, left_on='Species1_BirdLife', right_on='Species1_BirdLife', how='left'
)

In [125]:
all_combined_traits_df.to_csv('All_combined_eco_function_traits.csv',index=False)

In [126]:
all_combined_traits_df

,Species2_eBird,Common_Name1_eBird,ORDER1_eBird,FAMILY1_eBird,Species1_BirdLife,Species3_BirdTree,Beak_Length_Culmen,Beak_Length_Nares,Beak_Width,Beak_Depth,Tarsus_Length,Wing_Length,Kipps_Distance,Secondary1,Hand-Wing_Index,Tail_Length,Mass,Migration,Min_Latitude,Max_Latitude,Centroid_Latitude,Centroid_Longitude,Range_Size,Trophic_Level_Carnivore,Trophic_Level_Herbivore,Trophic_Level_Omnivore,Trophic_Level_Scavenger,Trophic_Niche_Aquatic predator,Trophic_Niche_Frugivore,Trophic_Niche_Granivore,Trophic_Niche_Herbivore aquatic,Trophic_Niche_Herbivore terrestrial,Trophic_Niche_Invertivore,Trophic_Niche_Nectarivore,Trophic_Niche_Omnivore,Trophic_Niche_Scavenger,Trophic_Niche_Vertivore,Primary_Lifestyle_Aerial,Primary_Lifestyle_Aquatic,Primary_Lifestyle_Generalist,Primary_Lifestyle_Insessorial,Primary_Lifestyle_Terrestrial,Realm_Indomalayan,Realm_Palearctic,Realm_Nearctic,Realm_Afrotropical,Realm_Oceanian,Realm_Neotropical,Realm_Australasian,Realm_Antarctic,redlistCategory,populationTrend,habitat_breadth,Diet_Inv,Diet_Vend,Diet_Vect,Diet_Vfish,Diet_Vunk,Diet_Scav,Diet_Fruit,Diet_Nect,Diet_Seed,Diet_PlantO,Diet_5CatFruiNect,Diet_5CatInvertebrate,Diet_5CatOmnivore,Diet_5CatPlantSeed,Diet_5CatVertFishScav,ForStrat_watbelowsurf,ForStrat_wataroundsurf,ForStrat_ground,ForStrat_understory,ForStrat_midhigh,ForStrat_canopy,ForStrat_aerial,Diet_Breadth,ForStrat_Breadth,GenerationLength,TotalInnovations,TotalInnovations,TotalFoodTypeInnovations,TotalTechnicalInnovations,InnovationYesOrNo,FoodTypeInnovationYesOrNo,TechnicalInnovationYesOrNo,BrainSize,RelativeBrainSize
0,Struthio camelus,Common Ostrich,Struthioniformes,Struthionidae (Ostriches),Struthio camelus,Struthio camelus,90.5,51.2,47.3,23.4,481.2,579.0,0.3,578.7,0.1,309.2,111000.0,1.0,-30.91,22.47,2.55,19.29,8595319.06,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,2.445436,10.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,30.0,50.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0,4.0,1.0,16.8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
1,Struthio molybdophanes,Somali Ostrich,Struthioniformes,Struthionidae (Ostriches),Struthio molybdophanes,Struthio camelus,89.2,52.0,46.7,23.6,466.7,510.5,0.3,510.2,0.1,291.7,111000.0,1.0,-3.61,11.62,5.53,43.23,1079016.70,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,3.0,1.0,2.744249,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
2,Rhea americana,Greater Rhea,Rheiformes,Rheidae (Rheas),Rhea americana,Rhea americana,86.5,35.6,27.2,17.2,308.0,604.5,0.3,604.2,0.1,62.0,23000.0,1.0,-41.17,-4.65,-21.37,-53.45,6537549.66,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,1.0,3.822235,20.0,10.0,10.0,0.0,0.0,0.0,20.0,0.0,20.0,20.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0,6.0,1.0,10.5,1.0,1.0,1.0,0.0,1.0,1.0,0.0,18.88,-0.401
3,Rhea pennata,Lesser Rhea,Rheiformes,Rheidae (Rheas),Rhea pennata,Rhea pennata,68.6,33.1,24.7,16.3,292.5,629.5,0.4,629.1,0.1,56.8,23900.0,1.0,-53.54,-35.63,-44.99,-69.08,660510.26,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,4.929461,20.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,30.0,40.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0,4.0,1.0,11.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
4,Rhea pennata,Lesser Rhea,Rheiformes,Rheidae (Rheas),Rhea tarapacensis,Rhea pennata,75.6,35.0,25.1,14.6,296.0,607.0,0.4,629.1,0.1,65.0,23900.0,1.0,-28.79,-16.21,-22.22,-67.75,424810.53,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...